This document present steel cube with size (10mm) x (10mm) x (10mm) under shear loading using FEniCSx as PDE solver and gmsh as meshing tool

In [1]:
# Scaled variable
# cube sizes
length = 1.0e-2
width = 1.0e-2
height = 1.0e-2

# phisical properties
ν = 0.3
E = 2.1e11

λ = ν*E/(1+ν)/(1-2*ν)
μ = E/2/(1+ν)

ρ = 7800.

# define oputput 
## 0 - with XDMF file
## 1 - with VTK file
## 2 - with pyvista
OUTPUT = 0

# elements order
ORDER = 2

import of all packages

In [2]:
import numpy as np
import ufl

from mpi4py import MPI
from petsc4py.PETSc import ScalarType

from dolfinx import mesh, fem, plot, io
from dolfinx.io import XDMFFile, gmshio

import gmsh

import pyvista

the main purpose of this example is usage of quadratic hexahedron as elements obtained with gmsh, since it is applicable for nonregular meshes

in following we create box usign OpenCASCADe kernel of gmsh

### pyvista doesn't show the right mesh if it has second order. Use VTK or XDMF output instead in this case

In [3]:
gmsh.initialize()

model = gmsh.model()
model.add("main_domain")
model.setCurrent("main_domain")

# Choose if Gmsh output is verbose
gmsh.option.setNumber("General.Terminal", 0)

# Set elements order to the specified one
gmsh.option.setNumber("Mesh.ElementOrder", ORDER)
# Set elements size
# gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 5) # uncomment to use for mesh refinement dependending from its surface curvature
gmsh.option.setNumber("Mesh.MeshSizeMax", 10e-3)
gmsh.option.setNumber("Mesh.MeshSizeMin", 5e-3)

# Set threads number for distrebuted meshing
# gmsh.option.setNumber("Mesh.MaxNumThreads3D", 4)

# Set mesh algorithm (default is Delaunay triangulation)
# see https://gmsh.info/doc/texinfo/gmsh.html#Choosing-the-right-unstructured-algorithm
gmsh.option.setNumber("Mesh.Algorithm3D", 3)

# gmsh.option.setNumber("Mesh.RecombinationAlgorithm",3)
# gmsh.option.setNumber("Mesh.Recombine3DAll",1)

# Set the usage of hexahedron elements 
gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 2)


box_tag = model.occ.add_box(-length/2, -width/2, -height/2 , length, width, height)

# Synchronize OpenCascade representation with gmsh model
model.occ.synchronize()

# Add physical marker for cells. It is important to call this function
# after OpenCascade synchronization
model.add_physical_group(dim=3, tags=[box_tag])

# Generate the mesh
# model.mesh.generate(2)
# model.mesh.recombine()
model.mesh.generate(dim=3)



# Create a DOLFINx mesh (same mesh on each rank)
msh, cell_markers, facet_markers = gmshio.model_to_mesh(model, MPI.COMM_SELF,0)
msh.name = "Box"
cell_markers.name = f"{msh.name}_cells"
facet_markers.name = f"{msh.name}_facets"

# Finalize gmsh to be able to use it again
gmsh.finalize()

if OUTPUT == 0:
    # write mesh to XDMF file
    with io.XDMFFile(msh.comm, "out/mesh.xdmf", "w") as file:
        file.write_mesh(msh)
        file.write_meshtags(cell_markers)
        msh.topology.create_connectivity(msh.topology.dim - 1, msh.topology.dim)
        file.write_meshtags(facet_markers)
elif OUTPUT == 1:
    # write mesh to VTK file
    with io.VTKFile(msh.comm, "out/mesh.pvd", "w") as vtk:
        vtk.write_mesh(msh)
elif OUTPUT == 2: 
    # visualize using pyvista
    # to use it, ipycanvas and ipyvtklink py needs to be installed
    # Just run in terminal:
    # ~# pip install ipyvtklink ipycanvas
    pyvista.start_xvfb()

    # Create plotter and pyvista grid
    p = pyvista.Plotter()
    topology, cell_types, geometry = plot.create_vtk_mesh(msh)
    grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

    actor_0 = p.add_mesh(grid, show_edges=True)#style="wireframe", color="k")

    p.show_axes()
    if not pyvista.OFF_SCREEN:
       p.show()
    else:
       figure_as_array = p.screenshot("mesh.png")



Create function space
to use with quadratic hexahedal elements quadrature scheme shoud be speciafied (commented lines), but these doesn't work, idk why

In [4]:
# P_q = ufl.FiniteElement("CG", msh.ufl_cell(), degree=ORDER, quad_scheme="default")
# V = FunctionSpace(msh, P_q)
V = fem.VectorFunctionSpace(msh, ("CG", ORDER))

Create boundary conditions:
For the left side (x=-length/2) we apply bonded constrains (u[0] = 0)
The right side (x = length/2) we load with kinematic loading (u[0] = -1mm)

In [5]:
def clamped_boundary(x):
    return np.isclose(x[0], -length/2)

def shifted_boundary(x):
    return np.isclose(x[0], length/2)

fdim = msh.topology.dim - 1
boundary_facets_left = mesh.locate_entities_boundary(msh, fdim, clamped_boundary)
boundary_facets_right = mesh.locate_entities_boundary(msh, fdim, shifted_boundary)

u_D_l = np.array([0.,0.,0.], dtype=ScalarType)
u_D_r = np.array([0.,0.,-1e-3], dtype=ScalarType)

bc_l = fem.dirichletbc(u_D_l, fem.locate_dofs_topological(V, fdim, boundary_facets_left), V)
bc_r = fem.dirichletbc(u_D_r, fem.locate_dofs_topological(V, fdim, boundary_facets_right), V)

Specify the linear elastic problem (from the example https://jsdokken.com/dolfinx-tutorial/chapter2/linearelasticity_code.html)

In [6]:
T = fem.Constant(msh, ScalarType((0., 0., 0.)))
ds = ufl.Measure("ds", domain=msh)

def epsilon(u):
    return ufl.sym(ufl.grad(u)) # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)
def sigma(u):
    return λ * ufl.nabla_div(u) * ufl.Identity(len(u)) + 2*μ*epsilon(u)

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
f = fem.Constant(msh, ScalarType((0., 0., 0.)))
a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
L = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) * ds

solving of the problem

In [7]:
problem = fem.petsc.LinearProblem(a, L, bcs=[bc_l, bc_r], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

Output

In [8]:
uh.name = "Deformation"

    
if OUTPUT == 0:
    # write mesh to XDMF file
    with io.XDMFFile(msh.comm, "out/deformation.xdmf", "w") as xdmf:
        xdmf.write_mesh(msh)
        xdmf.write_function(uh)
elif OUTPUT == 1:
    # write mesh to VTK file
    with io.VTKFile(msh.comm, "out/deformation.pvd", "w") as vtk:
        vtk.write_mesh(msh)
        vtk.write_function(uh)
elif OUTPUT == 2: 
    # visualize using pyvista
    # to use it, ipycanvas and ipyvtklink py needs to be installed
    # Just run in terminal:
    # ~# pip install ipyvtklink ipycanvas
    pyvista.start_xvfb()

    # Create plotter and pyvista grid
    p = pyvista.Plotter()
    topology, cell_types, geometry = plot.create_vtk_mesh(msh)
    grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

    # # Attach vector values to grid and warp grid by vector
    grid["u"] = uh.x.array.reshape((geometry.shape[0], 3))
    actor_0 = p.add_mesh(grid, style="wireframe", color="k")
    warped = grid.warp_by_vector("u", factor=1.5)
    actor_1 = p.add_mesh(warped, show_edges=True)
    p.show_axes()
    if not pyvista.OFF_SCREEN:
       p.show()
    else:
       figure_as_array = p.screenshot("deformation.png")
